In [1]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.widget-label { display: contents !important; }</style>"))
display(HTML("<style>.slider-container { margin: 12px !important; }</style>"))

In [2]:
from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

In [3]:
def get_absolute_isotherms(a, b, v_values, T_values):
    #given v (1d array) and T (Nd array) calculate N isotherm arrays 
    
    isotherms = []
    
    #N_A = 6.02214129e23 

    R = 0.082 * 1.01325 #atm L mol^-1 K^-1 #now in bar L mol^-1 K^-1
    
    for T in T_values:
        isot = []
        for v in v_values:
            p = R*T/(v - b) - (a/v**2)
            isot.append(p)
            
        isotherms.append(isot)
        
    return isotherms

In [4]:
def calculate_critic(a, b):
    
    """
        This function calculates the critic point 
        (p_c, v_c, T_c) from given a and b parameters of 
        the Van der Waals equation of state for real gases.
        
        :math:`(P + a \\frac{n^2}{V^2})(V - nb) = nRT`
        
        :math:`p_c = \\frac{a}{27 b^2}`
        :math:`v_c = 3b`
        :math:`T_c = \\frac{8a}{27 b k_B}`
        
   #Args:
   #    a: Term related with the attraction between particles in
   #    J m^3/ mol^2.\n
   #    b: Term related with the volume that is occupied by one 
   #    mole of the molecules in 10^-3 m^3/mol.\n
   #    
   #Returns:
   #    p_c: Critical pressure in Pa.\n
   #    v_c: Critical volume in m^3/mol.\n
   #    T_c: Critical tenperature in K.\n
        
    """
    
    if b == 0.0:
        return None
    
    k_B = 1.3806488e-23 #m^2 kg s^-2 K^-1
    N_A = 6.02214129e23 
    R = 0.082 * 1.01325 #atm L mol^-1 K^-1 #now in bar L mol^-1 K^-1
    
    p_c = a/27.0/(b**2)
    v_c = 3.0*b
    T_c = 8.0*a/27.0/b/R
    
    return p_c, v_c, T_c

In [5]:
def update_isotherms(change):
    
    obj = change.owner
    T_values = [0.95*T_c]
    #if obj is a_slider_114_001 or obj is b_slider_114_001:
    #    isotherms = get_absolute_isotherms(a_slider_114_001.value, b_slider_114_001.value, v_values, T_values)
    #    new_state.y =  isotherms
    #    current_state.y =  isotherms
    #    a_line.y = isotherms
    #    b_line.y = isotherms
    #    
    #    a_slider_114_002.value, b_slider_114_002.value = a_slider_114_001.value, b_slider_114_001.value
    #    a_slider_114_003.value, b_slider_114_003.value = a_slider_114_001.value, b_slider_114_001.value
    
    if obj is a_slider_114_003:
        #p_c, v_c, T_c = calculate_critic(a_slider_114_003.value, b_initial)
        #T_values = [0.95*T_c]
        v_values = np.linspace(1.2*b_initial, 10*v_c, 500)
        isotherms = get_absolute_isotherms(a_slider_114_003.value, b_initial, v_values, T_values)
        a_line.y = isotherms[0]
        
    elif obj is b_slider_114_004:
        #v_values = np.linspace(1.2*b_slider_114_004.value, 10*v_c, 500)
        #p_c, v_c, T_c = calculate_critic(a_initial, b_slider_114_004.value)
        #T_values = [0.95*T_c]
        v_values = np.linspace(1.2*b_slider_114_004.value, 10*v_c, 500)
        
        if b_slider_114_004.value == 0.0:
            v_values = np.linspace(0.0001, 10*v_c, 500)
        
        isotherms = get_absolute_isotherms(a_initial,
                                           b_slider_114_004.value,
                                           v_values,
                                           T_values)
        b_line.x = v_values
        b_line.y = isotherms[0]
        
    #p_c, v_c, T_c = calculate_critic(a_slider_114_003.value, b_slider_114_004.value)    
    #T_values = [0.95*T_c]
    #v_values = np.linspace(1.2*b_slider_114_004.value, 10*v_c, 500)

    if b_slider_114_004.value == 0.0:
        v_values = np.linspace(0.0001, 10*v_c, 500)        
    
    isotherms = get_absolute_isotherms(a_slider_114_003.value,
                                       b_slider_114_004.value,
                                       v_values,
                                       T_values)
    unique_isotherm.x = v_values
    unique_isotherm.y = isotherms[0]

In [6]:
def restart(a):
    
    a_slider_114_003.value, b_slider_114_004.value = a_initial, b_initial

In [7]:
def bar_to_atm(p_values):
    
    for i in range(len(p_values)):
        p_values[i] = np.array(p_values[i]) * 0.9869 #unit change from bar to atm
        
    return p_values

In [11]:
a_initial = 5.536 #L^2 bar/mol^2
b_initial = 0.03049 #L/mol

a, b = a_initial, b_initial

p_c, v_c, T_c = calculate_critic(a, b)

T_values = [0.95*T_c, T_c, 1.2*T_c]
v_values = np.linspace(1.2*b, 10*v_c, 500)
colors = ['#0079c4','#f09205','#21c400']

p_values = get_absolute_isotherms(a, b, v_values, T_values)
p_values = bar_to_atm(p_values)

#######################################
#######CREATE THE FIGURES##############
#######################################


fig_114_001 = bq.Figure(title='p vs v (Fixed T)',
                marks=[],
                axes=[],
                animation_duration=0, #500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=70, bottom=60, left=80, right=30),
                toolbar = True,
                layout = widgets.Layout(width='100%', height='500px')
    )

fig_114_002 = bq.Figure(title='',
                marks=[],
                axes=[],
                animation_duration=0, #500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=30, bottom=60, left=25, right=10),
                toolbar = True,
                layout = widgets.Layout(width='90%', height='40%')
    )

fig_114_003 = bq.Figure(title='',
                marks=[],
                axes=[],
                animation_duration=0, #500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=10, bottom=60, left=25, right=10),
                toolbar = True,
                layout = widgets.Layout(width='90%', height='40%')
    )

fig_114_004 = bq.Figure(title='',
                marks=[],
                axes=[],
                animation_duration=0, #500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=10, bottom=60, left=25, right=10),
                toolbar = True,
                layout = widgets.Layout(width='90%', height='40%')
    )

fig_114_005 = bq.Figure(title='p vs v (Fixed T)',
                marks=[],
                axes=[],
                animation_duration=0, #500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=70, bottom=60, left=80, right=30),
                toolbar = True,
                layout = widgets.Layout(width='100%', height='500px')
    )

#fig_114_006 = bq.Figure(title='',
#                marks=[],
#                axes=[],
#                animation_duration=0, #500,
#                #layout = widgets.Layout(align_self='center', width='75%'),
#                legend_location='top-right',
#                background_style= {'fill': 'white',  'stroke': 'black'},
#                fig_margin=dict(top=50, bottom=60, left=80, right=30),
#                toolbar = True,
#                layout = widgets.Layout(width='90%', height='300px')
#    )

scale_x = bqs.LinearScale(min = 0.0, max = max(v_values))
scale_y = bqs.LinearScale(min = 0, max = 2.0*p_c)

axis_x = bqa.Axis(scale=scale_x,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values = np.linspace(0, max(v_values), 5),
                #num_ticks=5,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='v (L/mol)',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y = bqa.Axis(
                scale=scale_y,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values = np.linspace(0, 2.0*p_c, 4),
                #num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p (atm)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

axis_x_no_ticks = bqa.Axis(scale=scale_x,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=0,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='v (L/mol)',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='15px')

axis_y_no_ticks = bqa.Axis(
                scale=scale_y,
                tick_format='.0f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=0,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p (atm)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='15px')

fig_114_001.axes = [axis_x, axis_y]
fig_114_002.axes = [axis_x_no_ticks, axis_y_no_ticks]
fig_114_003.axes = [axis_x_no_ticks, axis_y_no_ticks]
fig_114_004.axes = [axis_x_no_ticks, axis_y_no_ticks]
fig_114_005.axes = [axis_x, axis_y]
#fig_114_006.axes = [axis_x, axis_y]

#######################################
#THE MARKS!!!!!!!!!####################
#######################################

x_values = [ v_values for i in range(len(p_values))]
y_values = []
color_values = []
label_values = []

for i in range(len(p_values)):
    
    y_values.append(p_values[i])
    color_values.append(colors[i])
    label_values.append(str(T_values[i]))
#    
#new_state.x = x
#new_state.y = y
#new_state.colors = color_values
#new_state.labels = labels
#
#old_state.x = x
#old_state.y = y
#old_state.colors = color_values
#old_state.labels = labels

new_state = bqm.Lines(
                x = x_values, 
                y = y_values, 
                scales = {'x': scale_x, 'y': scale_y}, 
                opacities = [1.0 for elem in p_values],
                visible = True, #True, #t == '1.00',
                colors = color_values,
                labels = label_values,
)

old_state = bqm.Lines(
                x = x_values, 
                y = y_values, 
                scales = {'x': scale_x, 'y': scale_y}, 
                opacities = [1.0 for elem in p_values],
                visible = True, #True, #t == '1.00',
                colors = color_values,
                labels = label_values,
)

current_state = bqm.Lines(
                x = x_values[0], 
                y = y_values[0], 
                scales = {'x': scale_x, 'y': scale_y}, 
                opacities = [1.0 for elem in p_values],
                visible = True, #True, #t == '1.00',
                colors = color_values,
                labels = label_values,
)

a_line = bqm.Lines(
                x = x_values[0], 
                y = y_values[0], 
                scales = {'x': scale_x, 'y': scale_y}, 
                opacities = [1.0 for elem in p_values],
                visible = True, #True, #t == '1.00',
                colors = color_values,
                labels = label_values,
)

b_line = bqm.Lines(
                x = x_values[0], 
                y = y_values[0], 
                scales = {'x': scale_x, 'y': scale_y}, 
                opacities = [1.0 for elem in p_values],
                visible = True, #True, #t == '1.00',
                colors = color_values,
                labels = label_values,
)

ideal_isotherms = get_absolute_isotherms(0, 0, v_values, T_values)
ideal_isotherms = bar_to_atm(ideal_isotherms)

ideal_line = bqm.Lines(
                x = x_values, 
                y = ideal_isotherms, 
                scales = {'x': scale_x, 'y': scale_y}, 
                opacities = [1.0 for elem in p_values],
                visible = True, #True, #t == '1.00',
                colors = color_values,
                labels = label_values,
)

unique_isotherm = bqm.Lines(
                x = x_values[0], 
                y = y_values[0], 
                scales = {'x': scale_x, 'y': scale_y}, 
                opacities = [1.0],
                visible = True, #True, #t == '1.00',
                colors = [color_values[0]],
                labels = [label_values[0]],
)

fig_114_001.marks = [old_state]
fig_114_002.marks = [current_state]
fig_114_003.marks = [a_line]
fig_114_004.marks = [b_line]
fig_114_005.marks = [ideal_line, unique_isotherm]

#a_slider_114_001 = widgets.FloatSlider(
#    min=0.0,
#    max=2.0*a,
#    step=0.1,
#    value=a,
#    description='a',
#    disabled=False,
#    continuous_update=False,
#    orientation='horizontal',
#    readout=True
#)
#
#a_slider_114_001.observe(update_isotherms, 'value')
#
#b_slider_114_001 = widgets.FloatSlider(
#    min=0.0,
#    max=4.0*b,
#    step=0.001,
#    value=b,
#    description='b',
#    disabled=False,
#    continuous_update=False,
#    orientation='horizontal',
#    readout=True
#)
#
#b_slider_114_001.observe(update_isotherms, 'value')

a_slider_114_003 = widgets.FloatSlider(
    min=0.0,
    max=2.0*a,
    step=0.1,
    value=a,
    description='a',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    layout=widgets.Layout(width='90%')
)

a_slider_114_003.observe(update_isotherms, 'value')

#b_slider_114_002 = widgets.FloatSlider(
#    min=0.0,
#    max=4.0*b,
#    step=0.001,
#    value=b,
#    description='b',
#    disabled=True,
#    continuous_update=False,
#    orientation='horizontal',
#    readout=True
#)

#a_slider_114_003 = widgets.FloatSlider(
#    min=0.0,
#    max=2.0*a,
#    step=0.1,
#    value=a,
#    description='a',
#    disabled=True,
#    continuous_update=False,
#    orientation='horizontal',
#    readout=True
#)

b_slider_114_004 = widgets.FloatSlider(
    min=0.0,
    max=4.0*b,
    step=0.001,
    value=b,
    description='b',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    layout=widgets.Layout(width='90%')
)

b_slider_114_004.observe(update_isotherms, 'value')

return_button = widgets.Button(
    description='Reset',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Return to the original state',
)

return_button.on_click(restart)

top_block = widgets.HBox([], layout=widgets.Layout(width='100%', align_self='center'))
top_block.children = [widgets.VBox([fig_114_001], layout=widgets.Layout(width='33%', align_items='center')),
                      widgets.VBox([fig_114_002, fig_114_003, a_slider_114_003], layout=widgets.Layout(width='16%', height='500px', align_items='center', margin='40px 0 0 0')),
                      widgets.VBox([fig_114_002, fig_114_004, b_slider_114_004], layout=widgets.Layout(width='16%', height='500px', align_items='center', margin='40px 0 0 0')),
                      widgets.VBox([fig_114_005], layout=widgets.Layout(width='33%', align_items='center')),
                     ]

bottom_block = widgets.HBox([], layout=widgets.Layout(width='100%', height='60px', align_self='center'))
bottom_block.children = [widgets.VBox([widgets.HTMLMath(value=r"\( (p + \frac{a}{v^2})(v - b) = k_B T \)")], layout=widgets.Layout(width='33%', align_items='center')),
                         widgets.VBox([return_button], layout=widgets.Layout(width='33%', align_items='center')),
                         widgets.VBox([widgets.HTMLMath(value=r"\( p v = k_B T \)")], layout=widgets.Layout(width='33%', align_items='center'))
                         ]
#top_block = widgets.HBox([], layout=widgets.Layout(width='50%', align_self='center'))

#top_block.children = [fig_114_001, a_slider, b_slider, return_button]

main_block = widgets.VBox([], layout=widgets.Layout(width='100%', align_items='center'))

main_block.children = [top_block, bottom_block]
main_block